In [1]:
"""
1. Json 등 형식에 맞는 파일 불러오기
2. article text를 추출
3. stop word 적용
4. text rank 핵심 문장으로 요약
5. //konply에서 명사만 추출 ex) okt.nouns(string)
6. 키워드 추출 (ex: TF-IDF)
7. 아니면 핵심 문장간 유사도 비교
8. 합당한 문장에서 지명 대조
9. 추출한 지명 따로 저장
"""

'\n1. Json 등 형식에 맞는 파일 불러오기\n2. article text를 추출\n3. stop word 적용\n4. text rank 핵심 문장으로 요약\n5. //konply에서 명사만 추출 ex) okt.nouns(string)\n6. 키워드 추출 (ex: TF-IDF)\n7. 아니면 핵심 문장간 유사도 비교\n8. 합당한 문장에서 지명 대조\n9. 추출한 지명 따로 저장\n'

In [12]:
# 1. csv file 불러오기
import pandas as pd
csv_data = pd.read_csv('201910article.csv', delimiter=',', encoding='UTF-8')

# 행렬 개수 확인
print("데이터 개수 :",csv_data.shape)
print(csv_data.shape[0])
print()

#csv_data

데이터 개수 : (1526, 4)
1526



In [13]:
# 2. article 컬럼만 추출(원본), sentence(특수문자, stop word 적용)
article_list = csv_data.article
print(article_list[:10])
#article_list[0].strip()

0    옆집이 성매매 오피스텔 중 형량은 낮고 돈은 많이 번다“5년간 오피 운영하다 적발 ...
1               조국 법무부 장관이 1일 서울 여의도 국회 본회의장에서 열린 제...
2    “학교에 검찰 압수수색 나오기 사흘 전쯤 정경심 교수가 나에게 전화해 ‘혹시 압수수...
3               윤석열 검찰총장 © 1 황기선 기자          서울뉴스1 ...
4               제17호 태풍 타파로 강풍이 몰아치는 강원도 안인진 해변 자료사...
5               【서울뉴시스】 전진환 기자  윤석열 검찰총장이 지난달 30일 오...
6    황교안  자유한국당 대표 투쟁은 문희상 의장 민주당 또 그 2중대 3중대의 불법적 ...
7               사진순천경찰서          전남 순천 한 종합병원에서 몰래카...
8    경향신문 ㆍ“대통령에 대한 압박이자 인사권에 정면 도전” 비판ㆍ“이 정도 됐으면 물...
9               서울 서초구 대검찰청 2019101뉴스1 © 1 구윤성 기자  ...
Name: article, dtype: object


In [14]:
# 3_1. 특수문자 제거

# 소문자 변환
# re 라이브러리로 특수문자 제거

# 읽어온 데이터 확인 (json_body)

sentence = []

# 특수문자 제거 라이브러리
import re
for a in range(len(article_list)):
    sentence.append(article_list[a].strip())
    
for i in range(len(sentence)):
    sentence[i] = sentence[i].lower() # lower() : 모든 문자를 소문자로 변환
    sentence[i] = re.sub("(<br\s*/><br\s*/>)|(\-)|(\/)", ' ', sentence[i]) # 특수문자 제거
    sentence[i] = re.sub("[.;:!\'?,\"()|[\]]", ' ', sentence[i]) # 특수문자 제거

print("특수문자 제거 sentence[1]") # 모든 article을 sentence에 저장 후 특수문자 제거 및 (영어)소문자화 
#print(sentence[1])
print()

특수문자 제거 sentence[1]



In [15]:
# 3_2. 외부 Stop Word 리스트 불러오기 (stop word ranking 기반) 
# 불용어 처리 - stop_word 리스트 불러오기 & 전처리
stop_words = []
f = open("C:\Python\source\stop_word.txt", 'r', encoding='UTF-8')
while True:
    line = f.readline()
    if not line: break
    #print(line) #정상 출력 확인
    stop_words.append(line.rstrip('\n')) # 개행문자 제거 후 추가
f.close()

# 불러온 stop word 리스트 출력
#print(stop_words) 

In [16]:
# 3_3. 불러온 Stop Word 적용  
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

result = [] 
for w in sentence: 
    if w not in stop_words: 
        result.append(w) 

# 위의 4줄은 아래의 한 줄로 대체 가능
# result=[word for word in word_tokens if not word in stop_words]

# 불용어 처리 결과 출력
print("stop word 필터링 결과 (result)")
#print(result[0])
print()

stop word 필터링 결과 (result)



In [17]:
# 6 TF-IDF 모델 생성 및 그래프 생성
#!pip install sklearn
#!pip install scikit-learn
#!pip install scipy
from sklearn.feature_extraction.text import TfidfVectorizer
X = sentence
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(X)

print(tfidf_matrix.shape)

(1526, 124781)


In [18]:
# 특정 키워드에서 중복 요소 찾기
keyw = ['아프리카', '돼지', '야생', '방역', '멧돼지', '발생']
w_list0 = []
w_list1 = []
w_list2 = []
w_list3 = []
w_list4 = []
w_list5 = []
w_list6 = []
w_list7 = []
w_list8 = []

# 아프리카
for i in range(len(sentence)):
    if( sentence[i].find(keyw[0]) >= 0 ):
        w_list0.append(i)
# 돼지
for i in range(len(sentence)):
    if( sentence[i].find(keyw[1]) >= 0 ):
        w_list1.append(i)
# 야생
for i in range(len(sentence)):
    if( sentence[i].find(keyw[2]) >= 0 ):
        w_list2.append(i)
# 방역
for i in range(len(sentence)):
    if( sentence[i].find(keyw[3]) >= 0 ):
        w_list3.append(i)
# 멧돼지  
for i in range(len(sentence)):
    if( sentence[i].find(keyw[4]) >= 0 ):
        w_list4.append(i)
# 발생
for i in range(len(sentence)):
    if( sentence[i].find(keyw[5]) >= 0 ):
        w_list5.append(i)

print("아프리카 포함 인덱스",w_list0)
print("돼지 포함 인덱스",w_list1)
print("야생 포함 인덱스",w_list2)
print("방역 포함 인덱스",w_list3)
print("멧돼지 포함 인덱스",w_list4)

# 살처분, 확진, 보상, 신고, 야생
# 키워드 추출 리스트에서 중첩 인덱스 찾기

# 아프리카 + 돼지
u_list = list(set(w_list0).intersection(w_list1))
print("아프리카 + 돼지 인덱스 : ", u_list)

# (아프리카 + 돼지) + 야생
u_list = list(set(u_list).intersection(w_list2))
print("아프리카 + 돼지 + 야생 인덱스 : ", u_list)

# (돼지 + 방역 + 야생) + 

#비교대상 1251, 295

아프리카 포함 인덱스 [163, 166, 231, 275, 295, 386, 401, 536, 567, 868, 887, 917, 926, 946, 955, 1035, 1251, 1254, 1267, 1367, 1388, 1516]
돼지 포함 인덱스 [78, 163, 166, 189, 231, 275, 295, 386, 536, 666, 845, 868, 887, 917, 926, 942, 955, 1251, 1267, 1367, 1368, 1388, 1508, 1516]
야생 포함 인덱스 [62, 217, 231, 256, 295, 410, 431, 481, 536, 887, 918, 950, 955, 983, 1035, 1267, 1283, 1388, 1516]
방역 포함 인덱스 [25, 133, 163, 166, 231, 295, 384, 536, 557, 1251, 1267, 1368, 1388, 1516]
멧돼지 포함 인덱스 [163, 231, 295, 536, 887, 917, 926, 955, 1267, 1367, 1388, 1516]
아프리카 + 돼지 인덱스 :  [386, 163, 868, 1251, 166, 295, 231, 1388, 1516, 275, 1267, 917, 887, 536, 1367, 955, 926]
아프리카 + 돼지 + 야생 인덱스 :  [295, 231, 1388, 1516, 1267, 887, 536, 955]


In [19]:
f_article = []
for numm in u_list:
    f_article.append(sentence[numm])
    #print(sentence[numm])
    #print()
    
# 추출한 명사를 리스트로 만들기
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Kkma, Twitter, Hannanum, Komoran
from konlpy.utils import pprint

mydoclist = f_article

# twitter 와 komoran이 가장 높은 유사도를 보인다.
#kkma = Kkma()
#twitt = Twitter()
#hann = Hannanum()
komo = Komoran()

doc_nouns_list = [] 
 
for doc in mydoclist:
    nouns = komo.nouns(doc)
    doc_nouns = ''
 
    for noun in nouns:
        doc_nouns += noun + ' ' # 추출한 명사를 문장으로 합침
 
    doc_nouns_list.append(doc_nouns) # doc_nouns_list에 추가
 
#print(doc_nouns_list[1])
for arti in doc_nouns_list:
    print(arti)
    print()

포천 뉴스1 이상 휴 기자 일 경기 포천시 관인면 돼지 농장 아프리카 돼지 열병 의심 신고 접수 경기북부 축산업 비상 포천 지리 강원도 양성 판정 날 경우 의 경기 이남 지역 남 하와 수 때문 이다 포천 의심 신고 접수 농장 돼지 마리 대형 농장 이날 오전 시 돼지 마리 폐사 방역 당국 의심 축 신고 접수 포천 철원 돼지 농장 긴급 이동 제한 발효 전망 해당 농장 반경 돼지 농장 이내 농장 여 마리 돼지 사육 중 확진 판정 날 경우 해당 농장 반경 돼지 예방 살 처분 포천시 정부 통상 방역 시스템 강도 방역 체계 유지 의심 신고 접수 동요 분위기 동안 포천 관내 초소 곳 설치 방역 활동 육군 군단 군단 등 개 부대 여명 군 장병 방역 초소 시와 임무 수행 중 포천시 돼지 농장 곳 마리 사육 경기북부 최대 축산 도시 포천 방역 망이 경우 전면 강도 예방 살 처분 정책 정부 방역 대책 경기북부 돼지 매장 때문 정부 강화군 경기 김포시 파주시 돼지 살 처분 지금 파주 연천 김포 강화 등 비무장지대 민간인 통제 선 해상 북방한계선 지자체 발생 일 연천군 측 남방 한계선 전방 지점 야생 멧돼지 사체 발견 감염 상태 때문 북한 멧돼지 국내 바이러스 유입 원 지적 제기 정부 지난해 김포 등 최 전방 지대 해안 철책 철거 파주 등 여개 초소 양 돈 인 5월 북한 발생 바이러스 멧돼지 철책 사이 유입 것 의구심 제기 정부 감염 경로 파악 상태 다 포천 북한 접경 지역 의심 신고 접수 관인면 돼지 농장 비교 북한 양성 판정 날 경우 포천 전역 위험 지대 경기 남부 지역 남하 강원도 확산 위 섬 다 최 접경 지역 곳 지난달 일 양주시 의심 신고 접수 바 음성 판정 바 있다 

앵커 야생 멧돼지 아프리카 돼지 열병 게 확인 그동안 멧돼지 신경 당국 비판 멧돼지 열병 동유럽 체코 사례 참고 필요 말 김 관 진 기자 입 니다 기자 지난 5월 북한 아프리카 돼지 열병 발병 환경부 접경 지 돼지 농가 주변 울타리 설치 등 차단 방역 적극 정작 국내 확진 사례 후 환경부 멧돼지 소극 대응 

In [20]:
# LSTM 분류
# 출처 : https://bangseogs.tistory.com/96


In [21]:
# 6 TF-IDF 모델 (유사도 분석)
# 참조링크 : 
from sklearn.feature_extraction.text import TfidfVectorizer
 
mydoclist = doc_nouns_list
 
tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(mydoclist)
 
document_distances = (tfidf_matrix * tfidf_matrix.T)
 
print ('유사도 분석을 위한 ' + str(document_distances.get_shape()[0]) + 'x' + str(document_distances.get_shape()[1]) + ' matrix')

print(document_distances[0]) # 요기서 0.2 넘는 데이터를 추린다.
#print(document_distances.toarray())

#  -   doc1 doc2 doc3
#doc1   1
#doc2        1
#doc3             1

유사도 분석을 위한 8x8 matrix
  (0, 3)	0.06485271918433826
  (0, 1)	0.1603533049143991
  (0, 7)	0.14747832556887527
  (0, 4)	0.10731096312091541
  (0, 5)	0.09123771151345962
  (0, 6)	0.4731250861989176
  (0, 2)	0.11729945885237755
  (0, 0)	0.9999999999999998


In [22]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

ddoc1= []
ddoc1 = twitt.nouns(doc_nouns_list[0])

ddoc2 = []
ddoc2 = twitt.nouns(doc_nouns_list[1])
#
print(ddoc1)
print()
print(ddoc2)
#print(cos_sim(ddoc1, ddoc2))



NameError: name 'twitt' is not defined

In [23]:
# 문장의 유사도 분석하기(레벤슈타인 거리, N-gram)
# 참조 : https://github.com/suites/actual-deeplearning/blob/master/ch6/lev-distance.py
def calc_distance(a, b):
    if a == b:
        return 0
    a_len = len(a)
    b_len = len(b)
    if a == "":
        return b_len
    if b == "":
        return a_len
    # 2차원 표 (a_len+1, b_len+2) 준비하기
    matrix = [[] for i in range(a_len+1)]
    for i in range(a_len+1):
        matrix[i] = [0 for j in range(b_len+1)]
    # 0일 때 초기값을 설정
    for i in range(a_len+1):
        matrix[i][0] = i
    for j in range(b_len+1):
        matrix[0][j] = j
    # 표 채우기
    for i in range(1, a_len+1):
        ac = a[i-1]
        for j in range(1, b_len+1):
            bc = b[j-1]
            cost = 0 if (ac == bc) else 1
            matrix[i][j] = min([
                matrix[i-1][j] + 1,     # 문자 삽입
                matrix[i][j-1] + 1,     # 문자 제거
                matrix[i-1][j-1] + cost # 문자 변경
            ])
    return matrix[a_len][b_len]

print(calc_distance(doc_nouns_list[0], doc_nouns_list[0]))

0


In [24]:
# 문장단위 토큰화
# https://soyoung-new-challenge.tistory.com/45


In [25]:
#LSTM 으로 기사 분류
# 참조 : https://mc.ai/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%ACnlp-17%EC%9D%BC%EC%B0%A8-%EB%A1%9C%EC%9D%B4%ED%84%B0-%EB%89%B4%EC%8A%A4-%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0/



In [38]:
#!pip install networkx

import networkx
import re
 
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            if len(ch) == 1:
                ch.append(".")
                
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0
 
 
    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])
 
    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
 
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
 
    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
 
        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)
 
        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)
 
        return both
 
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))


In [42]:
tr = TextRank()
print('Load...')
from konlpy.tag import Komoran
tagger = Komoran()
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
#tr.loadSents(doc_nouns_list[0], lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)))
tr.loadSents(RawTagger(doc_nouns_list[1]), lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)))
# RawTagger("특정 텍스트....")
#tr = doc_nouns_list[0]
print('Build...')
tr.build()
ranks = tr.rank()
for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
    
print(tr.summarize(0.2))

Load...
Build...
0	1.0	[('앵커', 'NNP'), ('야생', 'NNG'), ('멧돼지', 'NNP'), ('아프리카', 'NNP'), ('돼지', 'NNP'), ('열병', 'NNG'), ('게', 'NNG'), ('확인', 'NNG'), ('그동안', 'NNG'), ('멧돼지', 'NNP'), ('신경', 'NNG'), ('당국', 'NNG'), ('비판', 'NNG'), ('멧돼지', 'NNP'), ('열병', 'NNG'), ('동유럽', 'NNP'), ('체코', 'NNP'), ('사례', 'NNG'), ('참', 'VV'), ('고', 'EC'), ('필요', 'NNG'), ('말', 'NNG'), ('김', 'NNP'), ('관', 'NNG'), ('지', 'VX'), ('ㄴ', 'ETM'), ('기자', 'NNG'), ('입', 'NNG'), ('니다', 'NNP'), ('기자', 'NNG'), ('지나', 'VV'), ('ㄴ', 'ETM'), ('5월', 'NNP'), ('북한', 'NNP'), ('아프리카', 'NNP'), ('돼지', 'NNP'), ('열병', 'NNG'), ('발병', 'NNG'), ('환경부', 'NNP'), ('접경', 'NNG'), ('지', 'NNB'), ('돼지', 'NNP'), ('농가', 'NNP'), ('주변', 'NNG'), ('울타리', 'NNP'), ('설치', 'NNG'), ('등', 'NNB'), ('차단', 'NNP'), ('방역', 'NNP'), ('적극', 'NNG'), ('정작', 'NNG'), ('국내', 'NNG'), ('확진', 'NNG'), ('사례', 'NNG'), ('후', 'NNG'), ('환경부', 'NNP'), ('멧돼지', 'NNP'), ('소극', 'NNP'), ('대응', 'NNG'), ('집', 'NNG'), ('돼지', 'NNP'), ('잡기', 'NNP'), ('집중', 'NNG'), ('동안', 'NNG'), ('야생', 'NNG'), ('멧돼지'

In [65]:
top_key = []

tr = TextRank(window=5, coef=1)
print('Load...')
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV'), ('없', 'VV') ])
tr.load(RawTagger(doc_nouns_list[0]), lambda w: w not in stopword and (w[1] in ('NNG', 'NNP', 'VV', 'VA')))
print('Build...')
tr.build()
kw = tr.extract(0.1)
for k in sorted(kw, key=kw.get, reverse=True):
    print("%s\t%g" % (k, kw[k]))
    top_key.append("%s %g" % (k, kw[k]))


Load...
Build...
신고 접수 4.394449154672439
의심 신고 4.212127597878484
북한 멧돼지 3.7013019741124933
돼지 농장 3.0081547935525483
지역 의심 3.0081547935525483
포천 북한 2.720472721100767
경우 포천 2.720472721100767
정부 방역 2.63086056241108
포천 의심 2.315007612992603
접수 포천 2.315007612992603
포천 방역 2.1608569331653444
접수 농장 2.0918640616783932
농장 돼지 1.3987168811184478
(('돼지', 'NNP'), ('농장', 'NNP'))	0.207457
(('신고', 'NNP'), ('접수', 'NNG'))	0.182408
(('포천', 'NNP'), ('북한', 'NNP'))	0.115707
(('정부', 'NNG'), ('방역', 'NNP'))	0.111596
(('지역', 'NNG'), ('의심', 'NNG'))	0.10454
(('경기', 'NNP'),)	0.0641438
(('경우', 'NNG'),)	0.0616773
(('멧돼지', 'NNP'),)	0.0586824


In [66]:
print(kw[k])
print(k)
print(k[0][0])
print("top_key")
print(top_key) # article 인덱스별 상위 키워드 탑 키에서 돼지 열병 관련 키워드를 골라 인덱스 선별, 선별한 기사 목록에서 지명만 추출, 지명 파일로 저장

0.05868244039230347
(('멧돼지', 'NNP'),)
멧돼지
top_key
["(('돼지', 'NNP'), ('농장', 'NNP')) 0.207457", "(('신고', 'NNP'), ('접수', 'NNG')) 0.182408", "(('포천', 'NNP'), ('북한', 'NNP')) 0.115707", "(('정부', 'NNG'), ('방역', 'NNP')) 0.111596", "(('지역', 'NNG'), ('의심', 'NNG')) 0.10454", "(('경기', 'NNP'),) 0.0641438", "(('경우', 'NNG'),) 0.0616773", "(('멧돼지', 'NNP'),) 0.0586824"]


In [44]:
for arti in doc_nouns_list:
    tr = TextRank(window=5, coef=1)
    print('Load...')
    stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV'), ('없', 'VV') ])
    tr.load(RawTagger(arti), lambda w: w not in stopword and (w[1] in ('NNG', 'NNP', 'VV', 'VA')))
    print('Build...')
    tr.build()
    kw = tr.extract(0.1)
    for k in sorted(kw, key=kw.get, reverse=True):
        print("%s\t%g" % (k, kw[k]))
    
    print()

Load...
Build...
신고 접수 4.394449154672439
의심 신고 4.212127597878484
북한 멧돼지 3.7013019741124933
돼지 농장 3.0081547935525483
지역 의심 3.0081547935525483
포천 북한 2.720472721100767
경우 포천 2.720472721100767
정부 방역 2.63086056241108
포천 의심 2.315007612992603
접수 포천 2.315007612992603
포천 방역 2.1608569331653444
접수 농장 2.0918640616783932
농장 돼지 1.3987168811184478
(('돼지', 'NNP'), ('농장', 'NNP'))	0.207457
(('신고', 'NNP'), ('접수', 'NNG'))	0.182408
(('포천', 'NNP'), ('북한', 'NNP'))	0.115707
(('정부', 'NNG'), ('방역', 'NNP'))	0.111596
(('지역', 'NNG'), ('의심', 'NNG'))	0.10454
(('경기', 'NNP'),)	0.0641438
(('경우', 'NNG'),)	0.0616773
(('멧돼지', 'NNP'),)	0.0586824

Load...
Build...
사살 대책 4.174387269895637
아프리카 돼지 4.020236590068379
돼지 열병 3.8379150332744243
열병 발병 3.7689221617874726
니다 기자 3.7689221617874726
야생 멧돼지 3.5682514663253215
체코 사례 3.258096538021482
멧돼지 사살 2.875104285765376
체코 아프리카 2.747270914255491
열병 바이러스 2.5649493574615367
멧돼지 바이러스 1.9588135538912212
멧돼지 아프리카 1.9588135538912212
체코 멧돼지 1.9588135538912212
멧돼지 열병 1.7764919970972666
(('야생